In [1]:
import sys
import os

sys.path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "src"))
os.chdir(os.path.dirname(os.path.abspath(os.getcwd())))

import torch

from src.mslm.utils.setup_train import setup_paths
from src.mslm.utils import create_dataloaders, build_model, run_training, prepare_datasets, ConfigLoader

# Training

In [2]:
from torch.utils.data import Subset

_, _, h5_file = setup_paths()
device = "cuda" if torch.cuda.is_available() else "cpu"

model_parameters = ConfigLoader("config/model/config.toml").load_config()
model_parameters.update({
    "device": device if model_parameters.get("device") == "auto" else model_parameters.get("device", device),
    "input_size": 250 * 2,
    "output_size": 3072,
})

# --- parametros de entrenamiento ---
train_ratio = 0.8
epochs = 10
batch_size = 2
checkpoint_interval = 5
log_interval = 2

# --- config de entrenamiento ---
train_config = ConfigLoader("config/training/train_config.toml").load_config()
train_ratio = train_config.get("train_ratio", train_ratio)
train_config.update({
    "model_version": 1000,
    "learning_rate": train_config.get("learning_rate", 0.00238),
    "epochs": epochs if epochs else train_config.get("epochs", 100),
    "batch_size": batch_size if batch_size else train_config.get("batch_size", 32),
    "checkpoint_interval": checkpoint_interval if checkpoint_interval else train_config.get("checkpoint_interval", 5),
    "log_interval": log_interval if log_interval else train_config.get("log_interval", 2),
    "train_ratio": train_ratio,
    "validation_ratio": round(1 - train_ratio, 2),
    "device": device if model_parameters.get("device") == "auto" else model_parameters.get("device", device),
})
    
tr_ds, val_ds = prepare_datasets(h5_file, train_ratio)

# Seleccionamos solo los primeros 2 ítems de cada dataset para pruebas rápidas
train_subset = Subset(tr_ds, list(range(2)))
val_subset = Subset(val_ds, list(range(2)))

tr_dl, val_dl = create_dataloaders(train_subset, val_subset, batch_size, num_workers=4)

Train size:	10191
Validation size:	2547


In [3]:
for a in tr_dl:
    print(a[2].shape)

torch.Size([2, 4, 3072])


In [4]:
for a in train_subset:
    print(a[0].shape, a[1].shape)

torch.Size([88, 250, 2]) torch.Size([3, 3072])
torch.Size([118, 250, 2]) torch.Size([4, 3072])


In [5]:
model = build_model(**model_parameters)

/home/giorgio6846/Code/Sign-AI/Sign-Multimodal-Language-Model/.conda/lib/python3.11/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


OptimizedModule(
  (_orig_mod): Imitator(
    (linear_feat): Sequential(
      (0): Linear(in_features=500, out_features=512, bias=True)
      (1): GELU(approximate='none')
      (2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (3): Linear(in_features=512, out_features=256, bias=True)
      (4): GELU(approximate='none')
      (5): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (conv1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (act1): GELU(approximate='none')
    (conv2): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
    (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (act2): GELU(approximate='none')
    (linear_hidden): Linear(in_features=256, out_features=512, bias=True)
    (pe): PositionalEncoding(
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x Transfor

In [6]:
train_config.update({
  "epochs": 200,
})
run_training(train_config, tr_dl, val_dl, model)

cuda
Starting training...
LR: 0.0001


Entrenando:   0%|          | 0/200 [00:06<?, ?it/s]


Epoch: 0.	 Total loss: 2.811551809310913


Entrenando:   0%|          | 1/200 [00:08<29:06,  8.78s/it]('Grad tensors ["L['self'].param_groups[0]['params'][0].grad", "L['self'].param_groups[0]['params'][1].grad", "L['self'].param_groups[0]['params'][2].grad", "L['self'].param_groups[0]['params'][3].grad", "L['self'].param_groups[0]['params'][4].grad", "L['self'].param_groups[0]['params'][5].grad", "L['self'].param_groups[0]['params'][6].grad", "L['self'].param_groups[0]['params'][7].grad", "L['self'].param_groups[0]['params'][8].grad", "L['self'].param_groups[0]['params'][9].grad", "L['self'].param_groups[0]['params'][10].grad", "L['self'].param_groups[0]['params'][11].grad", "L['self'].param_groups[0]['params'][12].grad", "L['self'].param_groups[0]['params'][13].grad", "L['self'].param_groups[0]['params'][14].grad", "L['self'].param_groups[0]['params'][15].grad", "L['self'].param_groups[0]['params'][16].grad", "L['self'].param_groups[0]['params'][17].grad", "L['self'].param_groups[0]['params'][18].grad", "L['self'].param_groups

Validation loss: 3.216034173965454


...


Epoch: 66.	 Total loss: 0.3706081807613373
Validation loss: 3.116382360458374


Entrenando:  34%|███▍      | 68/200 [01:51<03:05,  1.41s/it]('Grad tensors ["L['self'].param_groups[0]['params'][0].grad", "L['self'].param_groups[0]['params'][1].grad", "L['self'].param_groups[0]['params'][2].grad", "L['self'].param_groups[0]['params'][3].grad", "L['self'].param_groups[0]['params'][4].grad", "L['self'].param_groups[0]['params'][5].grad", "L['self'].param_groups[0]['params'][6].grad", "L['self'].param_groups[0]['params'][7].grad", "L['self'].param_groups[0]['params'][8].grad", "L['self'].param_groups[0]['params'][9].grad", "L['self'].param_groups[0]['params'][10].grad", "L['self'].param_groups[0]['params'][11].grad", "L['self'].param_groups[0]['params'][12].grad", "L['self'].param_groups[0]['params'][13].grad", "L['self'].param_groups[0]['params'][14].grad", "L['self'].param_groups[0]['params'][15].grad", "L['self'].param_groups[0]['params'][16].grad", "L['self'].param_groups[0]['params'][17].grad", "L['self'].param_groups[0]['params'][18].grad", "L['self'].param_group


Epoch: 68.	 Total loss: 0.7478004097938538


Entrenando:  34%|███▍      | 68/200 [01:59<03:51,  1.75s/it]


RuntimeError: DataLoader worker (pid(s) 1251237, 1251238, 1251239, 1251240) exited unexpectedly

In [8]:
model.eval()
with torch.no_grad():
    for batch in val_dl:
        inputs, mask, targets, _ = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        mask = mask.to(device)
        outputs = model(inputs, mask)
        print(f"Inputs shape: {inputs.shape}, Outputs shape: {outputs.shape}, Targets shape: {targets.shape}")
        break  # Solo para probar el primer batch

RuntimeError: DataLoader worker (pid(s) 1251237, 1251238, 1251239, 1251240) exited unexpectedly

In [ ]:
L_common = min(outputs.size(1), targets.size(1))
pred_embs     = outputs   [:, :L_common]
target_embs   = targets [:, :L_common]
embedding_mask = mask[:, :L_common]

import torch.nn as nn

loss_fn = nn.MSELoss()
loss = loss_fn(pred_embs, target_embs)
print("MSE Loss:", loss.item())


MSE Loss: 2.086271286010742


In [ ]:
target_embs

tensor([[[ 0.0897, -0.3952, -0.1157,  ...,  0.4509,  0.3203,  0.3035],
         [ 0.9322, -1.1671, -0.2401,  ...,  0.9682, -0.9013,  0.0350],
         [ 0.1231, -0.4041, -0.9660,  ..., -0.4154, -2.0379,  1.2492],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0804, -0.4052, -0.1248,  ...,  0.4500,  0.3122,  0.2966],
         [ 0.0402, -2.1052,  1.2279,  ...,  1.7971, -0.5469, -0.7472],
         [-1.6225, -1.8591, -0.6591,  ..., -1.4079, -0.5445, -0.8075],
         ...,
         [-1.2914,  1.0705,  1.3059,  ..., -0.6800, -0.3671,  2.2190],
         [-0.8618,  1.6748,  2.6077,  ..., -0.8423,  0.7961,  1.5343],
         [ 0.2764, -1.0429, -2.7705,  ...,  0.1155, -0.4705, -0.9378]]],
       device='cuda:0')

In [ ]:
pred_embs

tensor([[[-0.1389, -0.5081, -0.3362,  ...,  0.3684,  0.2839,  0.4170],
         [-2.2625, -0.1232,  1.0554,  ...,  0.7423, -1.6623, -0.2631],
         [-0.7219,  1.0109,  0.8119,  ..., -0.0616, -3.9642, -1.5129],
         ...,
         [-1.9372, -0.5624,  0.1327,  ...,  0.5759, -2.5311, -0.8052],
         [-0.2607, -0.8605,  1.0719,  ...,  0.3578, -0.7986, -0.0767],
         [-1.3852, -0.7967,  0.2694,  ...,  0.5442, -1.5376, -1.2053]],

        [[-0.1372, -0.6028, -0.4260,  ...,  0.2968,  0.3841,  0.3483],
         [-2.1371, -0.1450,  0.8849,  ...,  0.6219, -1.4993, -0.1890],
         [-0.6464,  1.0247,  0.7741,  ..., -0.1103, -3.8964, -1.4245],
         ...,
         [-2.0894, -0.5207, -0.0071,  ...,  0.2462, -2.8643, -0.8986],
         [-0.0597, -0.9253,  1.1884,  ...,  0.0332, -0.6494, -0.0531],
         [-0.9783, -0.8845, -0.0953,  ...,  0.4618, -1.0861, -1.2029]]],
       device='cuda:0')

In [ ]:
outputs

tensor([[[-0.1389, -0.5081, -0.3362,  ...,  0.3684,  0.2839,  0.4170],
         [-2.2625, -0.1232,  1.0554,  ...,  0.7423, -1.6623, -0.2631],
         [-0.7219,  1.0109,  0.8119,  ..., -0.0616, -3.9642, -1.5129],
         ...,
         [-1.4484, -1.0098, -0.1699,  ..., -0.2217, -0.6729,  0.2320],
         [-0.4084, -0.0734, -0.1730,  ...,  0.3982, -0.6137,  0.0183],
         [-1.7066, -0.0720,  0.0958,  ...,  0.3263, -1.3347, -0.3726]],

        [[-0.1372, -0.6028, -0.4260,  ...,  0.2968,  0.3841,  0.3483],
         [-2.1371, -0.1450,  0.8849,  ...,  0.6219, -1.4993, -0.1890],
         [-0.6464,  1.0247,  0.7741,  ..., -0.1103, -3.8964, -1.4245],
         ...,
         [-1.6019, -0.9580, -0.2105,  ..., -0.2810, -0.2262,  0.0546],
         [-0.4252,  0.0775, -0.3439,  ...,  0.4071, -0.1987, -0.2586],
         [-1.8843,  0.0857,  0.0350,  ...,  0.2321, -1.1898, -0.6596]]],
       device='cuda:0')